# Business Case: Alocação de Cabinets de Bateria

O objetivo desta análise é propor uma estratégia para **otimizar a distribuição dos cabinets** de baterias entre as estações, com foco em melhorar a experiência de troca de baterias dos usuários e a eficiência operacional da rede.

Com base nos dados de **tráfego de motos elétricas próximas às estações** (observations) e no histórico de **trocas de bateria** (swaps), buscamos:

- Alocar **10 novos cabinets** nas estações com maior demanda reprimida ou sobrecarga;
- Identificar **estações subutilizadas** onde faz sentido remover cabinets para realocação;

Todas as features do conjunto de dados utilizado neste estudo (*stations_processed.csv*) foram calculadas durante a etapa de processamento dos dados. Para entendê-las mais a fundo, visite a documentação da pasta */src* do projeto.

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
df = pd.read_parquet('../data/processed/stations_processed.parquet', engine='pyarrow')

## Análise Descritiva Inicial

In [ ]:
print(f'Quantidade de estações do conjunto: {len(df)}')

In [ ]:
df.head()

In [ ]:
df.columns

Algumas estações possuem a contagem de trocas igual a 0, pois não tiveram nenhuma troca na semana do dia 10/02/2025 - possivelmente estavam desativadas. Como as métricas do estudo foram calculadas utilizando essa variável, vamos descartar essas estações. 

In [ ]:
df = df[df['swaps_count']>0][:].reset_index()
print(f'Quantidade de estações do conjunto após descarte: {len(df)}')

Agora vamos analisar as estatísticas das colunas.

In [ ]:
num_cols = [
    'nearest_station_distance_km', 'cabinet_number',
    'swaps_per_day_mean', 'swaps_per_day_median', 'swaps_per_day_max',
    'swaps_per_hour_mean', 'swaps_per_hour_median', 'swaps_per_hour_max',
    'obs_max', 'obs_min', 'obs_mean', 'obs_sum', 'obs_q75',
    'n_points_in_radius', 'swaps_count', 'swaps_per_observation',
    'observations_per_swaps', 'swaps_per_cabinet'
]

df[num_cols[:8]].describe().round(2)

In [ ]:
df[num_cols[8:]].describe().round(2)

Alguns insights importantes das estatísticas:
- As estações não tendem a ficar muito longe umas das outras, com uma média de 1.6km;
- A mediana de cabinets de bateria por estação é de 2;
- As estatísticas para colunas que envolvem o número de observações parecem ser bem instáveis;
- A média de trocas diárias por cabinet fica em torno de 22.

## Correlação de Variáveis

Para entender como distribuir os cabinets de forma mais inteligente entre as estações, é crucial que os comportamentos das variáveis do conjunto sejam compreendidos mais a fundo, em especial como elas se relacionam com o número de cabinets. Portanto, vamos destrinchar esse tópico.

In [ ]:
df.cabinet_number.value_counts()

Aqui, podemos ver que o número de cabinets não tende a ser maior que 4.

### Médias por nº de cabinets

A seguir, vamos agrupar os dados pelo número de cabinets, e analisar as médias para cada quantidade.

In [ ]:
df[num_cols[:8]].groupby('cabinet_number').mean().round(2)

In [ ]:
df[num_cols[8:] + ['cabinet_number']].groupby('cabinet_number').mean().round(2)

Ao calcular as médias das variáveis em função do número de gabinetes, observa-se uma tendência clara de crescimento para indicadores relacionados à atividade total da estação, como a **swaps_per_day_mean**.
Isso sugere que essas variáveis estão **positivamente correlacionadas com o número de gabinetes disponíveis**, refletindo que estações com mais gabinetes tendem a atender um maior volume de trocas, como expressado no gráfico abaixo: 

In [ ]:
fig = px.scatter(
    df, 
    y='swaps_per_day_mean', 
    x='cabinet_number', 
    trendline='ols'
)

fig.update_layout(
    title='Dispersão entre Nº de Cabinets x Média Diária de Swaps',
    yaxis_title='Média Diária de Swaps',
    xaxis_title='Número de Cabinets',
    height=500, width=900
)

fig.update_traces(
    marker=dict(
        size=7,               
        symbol='diamond',
        opacity=0.65,
        line=dict(width=1, color='black')
    )
)

fig.show()

Embora a correlação entre os indicadores e o número de cabinets não implique causalidade, essa variável se mostra um **parâmetro relevante** para orientar a análise e o ajuste das métricas associadas aos swaps nas estações.

Por outro lado, o indicador **swaps_per_cabinet**, que mede a eficiência individual de cada gabinete (média diária de swaps / nº de cabinets), não demonstra um padrão consistente em relação ao número total de gabinetes.
Isso indica que a simples adição de mais gabinetes não indica um aumento ou diminuição proporcional na utilização de cada gabinete, como pode ser observado no gráfico abaixo:

In [ ]:
fig = px.scatter(
    df, 
    y='swaps_per_cabinet', 
    x='cabinet_number', 
    trendline='ols'
)

fig.update_layout(
    title='Dispersão entre Nº de Cabinets x Swaps por Cabinet',
    yaxis_title='Swaps por Cabinet',
    xaxis_title='Número de Cabinets',
    height=500, width=900
)

fig.update_traces(
    marker=dict(
        size=7,               
        symbol='diamond',
        opacity=0.65,
        line=dict(width=1, color='black')
    )
)

fig.show()

Isso é interessante, pois era de se esperar que, com o aumento do número de cabinets, houvesse uma redução na quantidade média de swaps por cabinet, refletindo uma diluição da demanda entre mais gabinetes.
No entanto, a ausência desse padrão pode indicar uma **lógica de alocação já eficiente**, em que os cabinets adicionais são implantados justamente nas estações com maior demanda, mantendo a utilização média de cada gabinete em níveis consistentes.

## Correlações

Para tornar a análise mais objetiva, vamos examinar diretamente os valores das correlações entre as variáveis e o número de cabinets.

In [ ]:
df_num = df[num_cols]

corr_with_cabinet = df_num.corr(method='spearman')['cabinet_number'].drop('cabinet_number')

corr_with_cabinet = corr_with_cabinet.sort_values(ascending=False).round(3)

corr_df = corr_with_cabinet.reset_index()
corr_df.columns = ['Variable', 'Correlation']

fig = px.imshow(
    [corr_df['Correlation']],                    
    x=corr_df['Variable'],                      
    y=['Correlation with cabinet_number'],       
    color_continuous_scale='Blues',
    zmin=-1, zmax=1,
    text_auto='.2f'
)

fig.update_layout(
    title='Correlation of Features with cabinet_number',
    xaxis=dict(tickangle=45),
    height=300
)

fig.show()

In [ ]:
# Calcula a matriz de correlação
corr_matrix = df_num.corr(method='spearman')

# Cria um heatmap interativo
fig = px.imshow(
    corr_matrix,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    color_continuous_scale='Blues',
    zmin=-1, zmax=1,
    text_auto='.2f'
)

fig.update_layout(
    title='Correlation Matrix (Spearman)',
    xaxis=dict(tickangle=45),
    width=800,
    height=800
)

fig.show()

## Distribuição de alocações

Agora que já entendemos um pouco melhor o comportamento da nossa variável de decisão, e já temos nossas features calculadas, vamos adentrar o problema em questão: a alocação e remoção inteligente de cabinets. Para isso, estaremos trabalhando com as 5 seguintes features:

- **swaps_per_cabinet:** média diária de swaps / nº de cabinets
- **swaps_per_observation:** swaps totais / volume de tráfego
- **observation_per_swaps:** volume de tráfego / swaps totais
- **nearest_station_distance_km:** distância da estação mais próxima
- **cabinet_number:** número de cabinets (na última semana)

Antes de seguirmos com a otimização, é importante alinharmos os conceitos de negócio que as features **observation_per_swaps** e **swaps_per_observations** representam.

**Interpretação dos Indicadores**

O indicador **swaps_per_observation** é definido como:

$$
\text{swaps\_per\_observation} = \frac{\text{swaps\_count}}{\text{observations}}
$$

Ele mede a eficiência da estação em converter o tráfego de motos próximas em swaps. Valores altos indicam que, mesmo com pouco tráfego, a estação consegue realizar muitas trocas, o que sugere alta eficiência e demanda. Valores baixos indicam que há muito tráfego passando, mas poucas trocas acontecendo, o que pode revelar um potencial de subutilização da estação. Essa variável é positivamente correlacionada com o número de cabinets.

Por outro lado, o indicador **observations_per_swap** é definido como:

$$
\text{observations\_per\_swap} = \frac{\text{observations}}{\text{swaps\_count}}
$$

Este é essencialmente o inverso do anterior e representa quantos veículos observados passam pela estação para cada swap realizado. Valores altos indicam que, para cada troca, muitas motos passam pela estação sem utilizar o serviço, o que pode sugerir que a estação precisa de mais cabinets para atender à demanda ou está mal posicionada. Já valores baixos indicam que cada moto que passa tem alta probabilidade de realizar um swap, sugerindo que a estação é eficiente ou próxima da saturação. Esse indicador é negativamente correlacionada com o número de cabinets.

Essas duas métricas são, portanto, inversamente proporcionais: quando uma é alta, a outra tende a ser baixa. Em geral, quando o **swaps_per_observation** é baixo e o **observations_per_swap** é alto, significa que há grande tráfego mas poucas trocas, sinalizando oportunidade de melhoria com a adição de mais cabinets. Por outro lado, quando o **swaps_per_observation** é alto e o **observations_per_swap** é baixo, isso quer dizer que a estação já opera de maneira eficiente.

**Otimização**

Agora que já entedemos os conceitos envolvidos na otimização, o critério de alocação de cabinets foi calculado a partir dos seguintes pesos atribuídos a cada variável:

- **w_swaps_cabinet = 35%**  
  Prioriza a eficiência ou saturação dos gabinetes já instalados, favorecendo estações que fazem mais swaps por gabinete.

- **w_obs_per_swaps = 30%**  
  Destaca locais com tráfego alto e poucas trocas, sugerindo potencial para crescimento com mais gabinetes.

- **w_swaps_per_obs = 15%**  
  Mede a capacidade de converter tráfego em swaps. Indica quão bem a estação atende à demanda que passa por ela.

- **w_distance = 10%**  
  Atua como leve penalização logística, desfavorecendo locais muito perto de outras estações.

- **w_cabinet_number = 10%**  
  Penaliza de forma sutil estações que já possuem muitos gabinetes, evitando subutilização.

**Estratégia Priorizada**

Essa configuração favorece:
- **Eficiência**: dá mais valor a estações que já operam bem (ou que possuem cabinets saturados).  
- **Oportunidade de expansão**: identifica locais onde há demanda reprimida.  
- **Equilíbrio operacional**: desprioriza alocação em estações muito próxima de outras ou com excesso de cabinets.

### Alocação de Cabinets

In [ ]:
# Função auxiliar para normalizar valores
def normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-9)

In [ ]:
def suggest_cabinets_allocation(df, cabinets_to_add=10, max_cabinets=8, exponent=5):
    df_alloc = df[df['cabinet_number']<max_cabinets].reset_index(drop=True).copy()

    # Normalização dos scores
    df_alloc['score_swaps_cabinet'] = normalize(df_alloc['swaps_per_cabinet'])
    df_alloc['score_swaps_per_obs'] = normalize(df_alloc['swaps_per_observation'])
    df_alloc['score_obs_per_swaps'] = normalize(df_alloc['observations_per_swaps'])
    df_alloc['score_distance'] = normalize(df_alloc['nearest_station_distance_km'])
    df_alloc['score_cabinet_number'] = 1 / (1 + df_alloc['cabinet_number'])
    
    # Pesos individuais
    w_swaps_cabinet = 0.35
    w_swaps_per_obs = 0.15
    w_obs_per_swaps = 0.3
    w_distance = 0.1
    w_cabinet_number = 0.1
    
    # Score de prioridade final
    df_alloc['priority_score_raw'] = (
          w_swaps_cabinet * df_alloc['score_swaps_cabinet']
        + w_swaps_per_obs * df_alloc['score_swaps_per_obs']
        + w_obs_per_swaps * df_alloc['score_obs_per_swaps']
        + w_distance * df_alloc['score_distance']
        + w_cabinet_number * df_alloc['score_cabinet_number']
    )

    # Aplicação de potência para maior diferenciação
    df_alloc['priority_score'] = df_alloc['priority_score_raw'] ** exponent
    
    # Alocação dos novos gabinetes
    df_alloc['allocation_float'] = (df_alloc['priority_score'] / df_alloc['priority_score'].sum()) * cabinets_to_add
    df_alloc['allocation'] = np.floor(df_alloc['allocation_float'])
    
    # Distribuir o restante (depois do floor)
    remaining = cabinets_to_add - int(df_alloc['allocation'].sum())
    if remaining > 0:
        top_up = df_alloc.sort_values('allocation_float', ascending=False).head(remaining).index
        df_alloc.loc[top_up, 'allocation'] += 1

    # Estimativa de ganho
    df_alloc['new_swaps_per_day_mean'] = df_alloc['swaps_per_day_mean'] + \
        df_alloc['swaps_per_cabinet'] * np.log1p(df_alloc['allocation']) / np.log1p(df_alloc['cabinet_number'] + 1)
    
    df_alloc['gain'] = df_alloc['new_swaps_per_day_mean'] - df_alloc['swaps_per_day_mean']

    # Organização do dataframe com os resultados
    cols = [
        'swap_station_id', 'allocation', 'cabinet_number', 
        'swaps_per_day_mean', 'new_swaps_per_day_mean', 'gain',
        'swaps_per_cabinet', 'swaps_per_observation', 
        'observations_per_swaps', 'nearest_station_distance_km']
    df_alloc = (
        df_alloc[df_alloc['allocation']>0][cols]
        .sort_values('allocation', ascending=False)
        .round(3)
        .reset_index(drop=True)
    )

    return df_alloc

In [ ]:
df_alloc = suggest_cabinets_allocation(df)
df_alloc

In [ ]:
gain_sum = df_alloc.gain.sum().round(2)
gain_mean = df_alloc.gain.mean().round(2)
print(f'Com a alocação sugerida acima, estima-se um ganho total de {gain_sum} swaps diários, com uma média de ganho de {gain_mean} swaps diários por estação.')

As alocações sugeridas pelo modelo indicam que algumas estações receberão múltiplos cabinets, como a estação 1451 (+3) e 446 (+2), enquanto a maioria receberá apenas 1 gabinete adicional. O ganho total estimado em swaps diários é significativo, refletindo o aumento da capacidade de atendimento nas estações selecionadas.

Estabelecimentos com menor número inicial de cabinets tendem a apresentar ganhos proporcionais maiores, evidenciando que a adição de cabinets nestas estações é eficiente. Já estações com cabinets já alocados continuam apresentando aumento relevante, mas o efeito relativo diminui possivelmente devido à saturação.

O indicador swaps_per_cabinet mostra que algumas estações são altamente eficientes individualmente, o que reforça a importância de direcionar cabinets adicionais para locais onde cada unidade terá impacto relevante. Já os indicadores swaps_per_observation e observations_per_swaps ajudam a identificar estações com grande demanda não atendida e potencial de melhoria.

### Remoção de Cabinets

In [ ]:
def suggest_cabinets_removal(df, cabinets_to_remove=10, min_cabinets=1, min_cabinets_remaining=0):
    df_rem = df[df['cabinet_number'] > min_cabinets].copy()

    # Normalizações
    df_rem['score_swaps_cabinet'] = normalize(df_rem['swaps_per_cabinet'])
    df_rem['score_swaps_per_obs'] = normalize(df_rem['swaps_per_observation'])
    df_rem['score_obs_per_swaps'] = normalize(df_rem['observations_per_swaps'])
    df_rem['score_distance'] = normalize(df_rem['nearest_station_distance_km'])
    df_rem['score_cabinet_number'] = 1 / (1 + df_rem['cabinet_number'])

    # Pesos
    w_swaps_cabinet = 0.35
    w_swaps_per_obs = 0.15
    w_obs_per_swaps = 0.3
    w_distance = 0.1
    w_cabinet_number = 0.1

    # Priority score “menos eficiente”
    df_rem['priority_score_raw'] = (
          w_swaps_cabinet * df_rem['score_swaps_cabinet']
        + w_swaps_per_obs * df_rem['score_swaps_per_obs']
        + w_obs_per_swaps * df_rem['score_obs_per_swaps']
        + w_distance * df_rem['score_distance']
        + w_cabinet_number * df_rem['score_cabinet_number']
    )
    df_rem['priority_score'] = df_rem['priority_score_raw'] ** 5

    # Score invertido para remoção
    df_rem['inverse_score'] = 1 / (df_rem['priority_score'] + 1e-9)

    # Distribuição inicial proporcional
    df_rem['removal_float'] = (df_rem['inverse_score'] / df_rem['inverse_score'].sum()) * cabinets_to_remove
    df_rem['removal'] = np.floor(df_rem['removal_float']).astype(int)

    # -------- Redistribuição iterativa para respeitar min_cabinets --------
    while df_rem['removal'].sum() < cabinets_to_remove:
        remaining = cabinets_to_remove - df_rem['removal'].sum()
        # Estaçoes que ainda podem perder cabinets
        df_rem['max_removable'] = df_rem['cabinet_number'] - min_cabinets - df_rem['removal']
        candidates = df_rem[df_rem['max_removable'] > 0]
        if candidates.empty:
            break  # não há mais onde remover
        # Ordenar por inverso do score (menos eficiente primeiro)
        candidates = candidates.sort_values('inverse_score', ascending=False)
        for idx in candidates.index:
            if remaining == 0:
                break
            df_rem.at[idx, 'removal'] += 1
            remaining -= 1

    # Garantir que não violamos min_cabinets
    df_rem['removal'] = df_rem.apply(
        lambda row: min(row['removal'], max(row['cabinet_number'] - min_cabinets_remaining, 0)), axis=1
    )

    # Estimativa de perda
    df_rem['new_swaps_per_day_mean'] = df_rem['swaps_per_day_mean'] - \
        df_rem['swaps_per_cabinet'] * np.log1p(df_rem['removal'].abs()) / np.log1p(df_rem['cabinet_number'])
    df_rem['loss'] = df_rem['swaps_per_day_mean'] - df_rem['new_swaps_per_day_mean']

    # Organização final
    cols = [
        'swap_station_id', 'removal', 'cabinet_number', 
        'swaps_per_day_mean', 'new_swaps_per_day_mean', 'loss',
        'swaps_per_cabinet', 'swaps_per_observation', 
        'observations_per_swaps', 'nearest_station_distance_km']
    
    df_rem = (
        df_rem[df_rem['removal']>0][cols]
        .sort_values('removal', ascending=False)
        .round(3)
        .reset_index(drop=True)
    )
    
    return df_rem

In [ ]:
df_rem = suggest_cabinets_removal(df)

In [ ]:
df_rem

In [ ]:
loss_sum = df_rem.loss.sum().round(2)
loss_mean = df_rem.loss.mean().round(2)
print(f'Com a alocação sugerida acima, estima-se uma perda total de {loss_sum} swaps diários, com uma média de perda de {loss_mean} swaps diários por estação.')

O modelo sugere remover cabinets de estações com menor demanda relativa ou excesso de cabinets, priorizando eficiência operacional. Por exemplo, 1411 e 1038 perderiam 2 cabinets cada, enquanto outras estações perderiam apenas 1.

Observa-se que, mesmo com a remoção, o impacto estimado nos swaps diários é muito baixo, comprovando a lógica de otimização e refletindo que essas estações possuem swaps_per_cabinet realmente baixos, indicando que cada cabinet não está sendo totalmente eficiente. Ao reduzir cabinets, conseguimos evitar subutilização e liberar recursos para estações com maior demanda.

Além disso, muitas das estações selecionadas para remoção apresentam swaps_per_observation baixos e observations_per_swaps relativamente altos, ou seja, há tráfego passando, mas cada swap exige mais observações, reforçando que a alocação extra de cabinets não seria eficiente. Muitas estações também possuem distância da estação mais próxima mínima, garantindo que a remoção não deixará áreas desassistidas.

Essa estratégia permite realocar cabinets para onde são mais necessários, aumentando a eficiência global e evitando desperdício em estações com baixa utilização.